In [2]:
dfc = sqlContext.read.load('file:////home/cloudera/Downloads/INPUT(2).csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [7]:
dfc.show()

+----------------+--------+--------------+--------+------------+
|User_Screen_Name|Verified|Follower_Count|Language|Tweets_Count|
+----------------+--------+--------------+--------+------------+
|    JP_Tisserand|   False|          1154|      fr|        1665|
|       marieecha|   False|          2321|      en|       83330|
|  paris_insoumis|   False|          2241|      fr|       10699|
|     Judiedixon3|   False|           540|      en|       28217|
|   mariama4niang|   False|           600|      fr|       26004|
|          jdpt62|   False|           103|      fr|          53|
| ricardotejada19|   False|           570|      es|       13181|
|   DanielPilotte|   False|         13196|      fr|      638764|
|        lmt93zoo|   False|          1090|      fr|       20414|
|        pat00005|   False|            17|      en|           4|
|      SeylliBens|   False|           501|      fr|       22200|
|       kanelle77|   False|          2433|      fr|       96613|
|     fch29421033|   Fals

In [24]:
dfc.registerAsTable('train_table')
df12 = sqlContext.sql('SELECT * from train_table WHERE Follower_Count > 4000 AND Verified = "True" ORDER BY Follower_Count desc')
df12.show(10)

/usr/lib/spark/python/pyspark/sql/dataframe.py:142: UserWarning: Use registerTempTable instead of registerAsTable.
  warnings.warn("Use registerTempTable instead of registerAsTable.")


+----------------+--------+--------------+--------+------------+
|User_Screen_Name|Verified|Follower_Count|Language|Tweets_Count|
+----------------+--------+--------------+--------+------------+
|       Le_Figaro|    True|       2619517|      fr|      316441|
|       Le_Figaro|    True|       2619516|      fr|      316440|
|       Le_Figaro|    True|       2619453|      fr|      316437|
|           afpfr|    True|       2617129|      fr|      291518|
|            libe|    True|       2492213|      fr|      121959|
|       20Minutes|    True|       2238648|      fr|      202479|
|       20Minutes|    True|       2237885|      fr|      202403|
|           BFMTV|    True|       2150453|      fr|      218543|
|           BFMTV|    True|       2150452|      fr|      218538|
|           BFMTV|    True|       2150451|      fr|      218539|
+----------------+--------+--------------+--------+------------+
only showing top 10 rows



In [16]:
df12.registerAsTable('train_table1')
df1 = sqlContext.sql('SELECT User_Screen_Name, MAX(Follower_Count) - MIN(Follower_Count) AS FOLLOWER_COUNT_INCREASE from train_table1 GROUP BY User_Screen_Name')

/usr/lib/spark/python/pyspark/sql/dataframe.py:142: UserWarning: Use registerTempTable instead of registerAsTable.
  warnings.warn("Use registerTempTable instead of registerAsTable.")


+----------------+-----------------------+
|User_Screen_Name|FOLLOWER_COUNT_INCREASE|
+----------------+-----------------------+
| michaelianblack|                      0|
|           Nagui|                     15|
|   JJBourdin_RMC|                      0|
|    salzburg_com|                      0|
|  timothycsimons|                      0|
|     jeangeorges|                      0|
|           rubin|                      6|
|      RenaudDely|                      0|
| YasmineBouchene|                      0|
|         Voodart|                      0|
|  EmmanuelMacron|                      0|
| julienmielcarek|                      0|
|       Midilibre|                      0|
|         JBollee|                      9|
|      senateur61|                      0|
|   RachelDonadio|                      0|
| Marion_M_Le_Pen|                      0|
| jleonardelli_fn|                      0|
|       manjusrii|                      0|
|           CNEWS|                      0|
+----------

In [19]:
df1 = df1.filter(df1["FOLLOWER_COUNT_INCREASE"] > 0)

root
 |-- User_Screen_Name: string (nullable = true)
 |-- FOLLOWER_COUNT_INCREASE: integer (nullable = true)



In [38]:
df1.registerAsTable('train_tableFin')
df1 = sqlContext.sql('SELECT * FROM train_tableFin ORDER BY FOLLOWER_COUNT_INCREASE DESC')
df1.count()

/usr/lib/spark/python/pyspark/sql/dataframe.py:142: UserWarning: Use registerTempTable instead of registerAsTable.
  warnings.warn("Use registerTempTable instead of registerAsTable.")


52

In [41]:
df1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("FOLLOWER_COUNT_INCREASE.csv")

In [28]:
sqlContext.sql('SELECT * FROM train_table1 ORDER BY Tweets_Count DESC').registerAsTable('table_Pop')

/usr/lib/spark/python/pyspark/sql/dataframe.py:142: UserWarning: Use registerTempTable instead of registerAsTable.
  warnings.warn("Use registerTempTable instead of registerAsTable.")


In [36]:
pop_users_tweet = sqlContext.sql('SELECT User_Screen_Name, MAX(Tweets_Count) AS TWEET_COUNT FROM table_Pop GROUP BY User_Screen_Name ORDER BY TWEET_COUNT DESC')

In [39]:
pop_users_tweet.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("TOP_USERS_TWEET_COUNT.csv")

In [37]:
pop_users_tweet.show(15)

+----------------+-----------+
|User_Screen_Name|TWEET_COUNT|
+----------------+-----------+
| NoticiasVenezue|    1956713|
|   Limportant_fr|     532949|
|       Le_Figaro|     316441|
|           afpfr|     291518|
|           BFMTV|     218543|
|       RTLFrance|     209051|
|       20Minutes|     202479|
|     le_Parisien|     194072|
|           CNEWS|     193156|
|         Europe1|     191482|
|    RadioFreeTom|     188210|
|            eNCA|     188183|
|      SputnikInt|     180648|
| LaRazon_Bolivia|     164665|
|      franceinfo|     163061|
+----------------+-----------+
only showing top 15 rows

